In [21]:
from flair.data import Sentence
from flair.models import SequenceTagger
import pandas as pd
import numpy as np
import time
import re
import multiprocessing as mp
import nltk
import nltk.data
nltk.download('punkt')

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#paths
pathBHR = '../archivos/csv/Bot_history_report.csv'
pathPrueba1 = '../data/BD_testparatesteo.xlsx'
pathPalabrasLlave = '../archivos/csv/Copia de Palabras llave RFP.csv'
pathTraining = '../archivos/csv/Training_data.csv'
pathOutput = '../out/out2.csv'
pathRes = '../out/res2.csv'
pathDur = '../out/dur2.csv'

In [3]:
#%% cositas de archivos
df = pd.read_excel(pathPrueba1)


In [32]:
df_testeo_10 = df[['DESCRIPCIONSOLICITUD']].loc[0:10]
df_testeo_100 = df[['DESCRIPCIONSOLICITUD']].loc[0:100]
df_testeo_1000 = df[['DESCRIPCIONSOLICITUD']].loc[0:1000]
df_testeo_10000 = df[['DESCRIPCIONSOLICITUD']].loc[0:10000]

In [43]:
tagger = SequenceTagger.load("flair/ner-spanish-large") # load tagger

def predicter(text):
    sent_tokenize = nltk.sent_tokenize(text)
    flair_sentences = [Sentence(sen) for sen in sent_tokenize]
    flair_tagger = tagger.predict(flair_sentences) 
    return flair_tagger


2023-01-21 15:53:08,540 loading file C:\Users\samue\.flair\models\ner-spanish-large\models--flair--ner-spanish-large\snapshots\9d4671d2f345c1258f37a29ce2321067f2ed296e\pytorch_model.bin
2023-01-21 15:53:29,632 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


In [45]:
#computo 
testeo_100=[(predicter(texto)) for texto in df_testeo_100['DESCRIPCIONSOLICITUD']]

#testeo 100 - 3 min 48 seg // ganó vs paralelo

In [46]:
pool = mp.Pool(mp.cpu_count())

testeo_100_paralelo = pool.map(predicter, [sen for sen in df_testeo_100['DESCRIPCIONSOLICITUD']])

pool.close()

In [ ]:
for index, sen in enumerate(serie):
    sentence = Sentence(sen)
    tagger.predict(sentence) # predict NER tags
    
    for entity in sentence.get_spans('ner'): # iterate over entities and print
        f.write(str(index) + ',' +  str(entity.get_label("ner").value) + ',' + str(entity.text) + ',' + str(entity.start_position) + ',' +  str(entity.end_position) + ',' +  str(entity.get_label("ner").score) + '\n')
f.close()

endTime = time.time()
duration = endTime - startTime # get the execution time